## PingOne Overview

PingOne is a cloud-based identity and access management service that provides secure identity solutions for enterprises, enabling seamless authentication and authorization across applications and services.

Key Features:
* **Single Sign-On (SSO)** - Users authenticate once to access multiple applications
* **Multi-Factor Authentication (MFA)** - Enhanced security through additional verification methods
* **Adaptive Authentication** - Risk-based authentication policies based on user behavior and context
* **Universal Directory** - Centralized user management and profile synchronization
* **API Access Management** - OAuth 2.0 and OpenID Connect support for API security

## Amazon Bedrock Gateway Overview

Bedrock AgentCore Gateway provides customers a way to turn their existing APIs and Lambda functions into fully-managed MCP servers without needing to manage infra or hosting. Customers can bring OpenAPI spec or Smithy models for their existing APIs, or add Lambda functions that front their tools. Gateway will provide a uniform Model Context Protocol (MCP) interface across all these tools. Gateway employs a dual authentication model to ensure secure access control for both incoming requests and outbound connections to target resources. The framework consists of two key components: Inbound Auth, which validates and authorizes users attempting to access gateway targets, and Outbound Auth, which enables the gateway to securely connect to backend resources on behalf of authenticated users. Together, these authentication mechanisms create a secure bridge between users and their target resources, supporting both IAM credentials and OAuth-based authentication flows. Gateway supports MCP's Streamable HTTP transport connection.

More details on Amazon Bedrock AgentCore Gateway can be found at:
- https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main/01-tutorials/02-AgentCore-gateway
- https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway.html

## Learning Objective

PingOne can be used as an identity provider on AgentCore Identity to authorize consuming application's access to protected Amazon AgentCore Gateway resources. In this notebook we will explore the use of PingOne for inbound authentication with Amazon Bedrock Gateway.

## Tutorial Architecture

<figure style="width: 70%;">
    <img src="/images/gateway_auth_diagram.png" style="width: 70%; height: auto;" alt="Detailed AWS Cloud Architecture">
</figure>

## Prerequisites

- PingOne environment
- PingOne IAM rights to create new IAM resources and applications
- Python 3.10+
- AWS Credentials
- AWS IAM rights to create a new AgentCore Agent and Lambda
- Amazon Bedrock AgentCore SDK
- Strands Agents
- AWS region that supports Bedrock AgentCore. Refer https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agentcore-regions.html

## Learning Objective 1: Setup PingOne for use with AgentCore Gateway

### 1.1: Create a new Resource

1. Login to your PingOne environment
2. Select **Applications**, then **Resources**, and add a resource
3. Configure the resource:
    - For **Resource Name** and **Audience**, enter **agentcore_gateway**
    - For **Scopes**, add **mcp:invoke**

<figure>
    <img src="/images/gateway_resource_config.png">
</figure>

### 1.2: Create a new Application

1. Select **Applications**, then **Applications**, and add an application
2. Select **OIDC Web App** as the **Application Type**
3. Configure the application:
    - For **Application Name**, enter **Agent Orchestrator**
    - For **Token Auth Method**, select **Client Secret Basic**
    - For **Grant Type**, select **Client Credentials**
    - For **Resources**, add the **mcp:invoke** scope on the **agentcore_gateway** resource configured in step 1.1

<figure>
    <img src="/images/gateway_application_config.png">
</figure>

## Learning Objective 2: Setup and Deploy Lambda Target

### 2.1: Install Dependencies

In [ ]:
# Install required packages from requirements.txt
%pip install --force-reinstall -U -r requirements.txt --quiet

### 2.2: Configure Environment Variables

In [ ]:
import os

# PingOne Configuration - Replace with your values
os.environ['PINGONE_CLIENT_ID'] = 'YOUR_CLIENT_ID_VALUE'
os.environ['PINGONE_CLIENT_SECRET'] = 'YOUR_CLIENT_SECRET_VALUE'
os.environ['PINGONE_AUDIENCE'] = 'agentcore_gateway'
os.environ['PINGONE_SCOPE'] = 'mcp:invoke'
os.environ['PINGONE_DISCOVERY_URL'] = 'https://auth.pingone.your-region/your-environment/as/.well-known/openid-configuration'
os.environ['PINGONE_TOKEN_URL'] = 'https://auth.pingone.your-region/your-environment/as/token'

### 2.3: Discover AWS Environment Identity

In [ ]:
import boto3
import zipfile
import io
from botocore.exceptions import ClientError
from boto3.session import Session
import time

boto_session = Session()
sts = boto3.client('sts')
region = boto_session.region_name
account_id = sts.get_caller_identity().get("Account")
print(f"📍 Deploying to Account: {account_id} in Region: {region}")

### 2.4: Define Lambda Tool Logic

Create a Python file for the Lambda function code. Note how the tool name is retrieved from the `context` object and utilized within the function logic.

In [ ]:
%%writefile lambda_function.py
def lambda_handler(event, context):
    print(f"Event: {event}")
    print(f"Context: {context}")
    extended_tool_name = context.client_context.custom["bedrockAgentCoreToolName"]
    resource = extended_tool_name.split("___")[1]
    city = event.get("city")

    print(resource)
    print(city)
    if resource == "weather_check":
        return f"Weather in {city} is bright and sunny!"
    elif resource == "directions":
        return f"Take I5 south all the way to {city} downtown"

### 2.5: Package Lambda Function for Deployment

In [ ]:
lambda_client = boto3.client('lambda', region_name=region)
with zipfile.ZipFile('lambda_function.zip', 'w') as zip_file:
    zip_file.write('lambda_function.py', 'lambda_function.py')

with open('lambda_function.zip', 'rb') as zip_file:
    zip_content = zip_file.read()

### 2.6: Create Lambda Execution Role

In [ ]:
iam_client = boto3.client('iam', region_name=region)

trust_policy = """{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": { "Service": "lambda.amazonaws.com" },
            "Action": "sts:AssumeRole"
        }
    ]
}"""

policy = """{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": ["logs:CreateLogGroup", "logs:CreateLogStream", "logs:PutLogEvents"],
            "Resource": "arn:aws:logs:*:*:*"
        }
    ]
}"""

try:
    response = iam_client.create_role(
        RoleName='lambda-role',
        AssumeRolePolicyDocument=trust_policy
    )
    lambda_role_arn = response['Role']['Arn']
    print(f"Created new role: {lambda_role_arn}")
    print("Waiting 10s for new role to propagate...")
    time.sleep(10)
except iam_client.exceptions.EntityAlreadyExistsException:
    response = iam_client.get_role(RoleName='lambda-role')
    lambda_role_arn = response['Role']['Arn']
    print(f"Using existing role: {lambda_role_arn}")

iam_client.put_role_policy(
    PolicyDocument=policy,
    PolicyName="lambda-policy",
    RoleName="lambda-role"
)

### 2.7: Deploy Lambda Function

In [ ]:
try:
    response = lambda_client.create_function(
        FunctionName='m2m-pingone-lambda',
        Runtime='python3.12',
        Role=lambda_role_arn,
        Handler='lambda_function.lambda_handler',
        Code={'ZipFile': zip_content},
    )
    print(f"Successfully created function.")
except lambda_client.exceptions.ResourceConflictException:
    response = lambda_client.update_function_code(
        FunctionName='m2m-pingone-lambda',
        ZipFile=zip_content,
    )
    print(f"Function already exists. Updated code instead.")

response = lambda_client.get_function(FunctionName='m2m-pingone-lambda')
lambda_arn = response["Configuration"]["FunctionArn"]
print(f"Target Lambda ARN: {lambda_arn}")

## Learning Objective 3: Setup AgentCore Gatway

### 3.1: Create Amazon Bedrock AgentCore Gateway Role and Permissions

Establish a dedicated IAM role that permits the Bedrock Gateway service to specifically invoke your newly created Lambda function.

In [ ]:
iam_client = boto3.client('iam')

trust_policy = """{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock-agentcore.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
"""

# Create role with trust policy
try:
    response = iam_client.create_role(
        RoleName='bedrock-agent-lambda-role',
        AssumeRolePolicyDocument=trust_policy
    )
    role_arn = response['Role']['Arn']
    print(f"Created new role: {role_arn}")
except iam_client.exceptions.EntityAlreadyExistsException:
    # If it exists, just fetch the existing ARN
    response = iam_client.get_role(RoleName='bedrock-agent-lambda-role')
    role_arn = response['Role']['Arn']
    print(f"Role already exists, using: {role_arn}")

permission = """{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "lambda:InvokeFunction"
            ],
            "Resource": [
                "%s"
            ],
            "Effect": "Allow",
            "Sid": "InvokeFunction"
        }
    ]
}
"""% lambda_arn


# Add Lambda invoke policy
iam_client.put_role_policy(
    RoleName='bedrock-agent-lambda-role',
    PolicyName='lambda-invoke-policy',
    PolicyDocument=permission
)

role_arn = response['Role']['Arn']
print(f"Role ARN: {role_arn}")

### 3.2: Provision Amazon Bedrock AgentCore Gateway

Create the Gateway instance itself, configuring it to use Custom JWT authentication so it knows to trust tokens issued by your PingOne environment.

In [ ]:
import os

gateway_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=region,
)

gateway_name = "m2m-pingone-gateway"
existing_gateways = gateway_client.list_gateways().get('items', [])
existing_gateway = next((g for g in existing_gateways if g['name'] == gateway_name), None)

if existing_gateway:
    gateway_id = existing_gateway['gatewayId']
    detail_response = gateway_client.get_gateway(gatewayIdentifier=gateway_id)
    gateway_url = detail_response["gatewayUrl"]
    print(f"Using existing gateway: {gateway_id}")
else:
    auth_config = {
        "customJWTAuthorizer": {
            "allowedAudience": [os.environ["PINGONE_AUDIENCE"]],
            "discoveryUrl": os.environ["PINGONE_DISCOVERY_URL"],
            "allowedScopes": [os.environ["PINGONE_SCOPE"]]
        }
    }
    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn=role_arn,
        protocolType="MCP",
        authorizerType="CUSTOM_JWT",
        authorizerConfiguration=auth_config,
        description="Customer Support AgentCore Gateway",
    )
    gateway_id = create_response["gatewayId"]
    gateway_url = create_response["gatewayUrl"]
    print(f"Created new gateway: {gateway_id}")

print(f"Gateway URL: {gateway_url}")

## Learning Objective 4: Register Lambda Tools to Gateway

### 4.1: Define Tool Metadata and Schemas

In [ ]:
api_spec = [
    {
        "name": "weather_check",
        "description": "Check the weather for a given City",
        "inputSchema": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city you want to get weather for"
                }
            },
            "required": [
                "city"
            ]
        }
    },
    {
        "name": "directions",
        "description": "Search the web for directions to a city",
        "inputSchema": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city you want to get directions to"
                }
            },
            "required": [
                "city"
            ]
        }
    }
]

### 4.2: Connect Lambda Target to Gateway

Link your Lambda function to the Gateway, officially registering it as a "Target" that can be reached via the Gateway's MCP interface

In [ ]:
existing_targets = gateway_client.list_gateway_targets(
    gatewayIdentifier=gateway_id
).get('items', [])
existing_target = next((t for t in existing_targets if t['name'] == "LambdaUsingSDK"), None)

if existing_target:
    target_id = existing_target['targetId']
    print(f"✅ Target 'LambdaUsingSDK' already exists. Target ID: {target_id}")
else:
    lambda_target_config = {
        "mcp": {
            "lambda": {
                "lambdaArn": lambda_arn,
                "toolSchema": {"inlinePayload": api_spec},
            }
        }
    }
    create_target_response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name="LambdaUsingSDK",
        description="Lambda Target using SDK",
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=[{"credentialProviderType": "GATEWAY_IAM_ROLE"}],
    )
    target_id = create_target_response["targetId"]
    print(f"🚀 Successfully created new target. Target ID: {target_id}")

print(f"Target is ready for use in Gateway: {gateway_id}")

## Learning Objective 5: Use the tools made available through AgentCore Gateway in your agent

### 5.1: Get a token

In [ ]:
import requests
import json

def fetch_access_token():
    response = requests.post(
        os.environ["PINGONE_TOKEN_URL"],
        data={
            "grant_type": "client_credentials",
            "scope": os.environ["PINGONE_SCOPE"]
        },
        auth=(
            os.environ["PINGONE_CLIENT_ID"], 
            os.environ["PINGONE_CLIENT_SECRET"]
        ),
        headers={'Content-Type': 'application/x-www-form-urlencoded'}
    )
    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        return None
    return response.json().get('access_token')

access_token = fetch_access_token()

### 5.2: Decode it and see the contents. Verify "iss", "aud" and "scope".

In [ ]:
import base64
import json

def decode_jwt_token(token):
    parts = token.split('.')
    header = json.loads(base64.b64decode(parts[0] + '==').decode('utf-8'))
    payload = json.loads(base64.b64decode(parts[1] + '==').decode('utf-8'))
    return header, payload


header, payload = decode_jwt_token(access_token)
print(f"Issuer: {payload.get('iss')}")
print(f"Audience: {payload.get('aud')}")
print(f"Scope: {payload.get('scope')}")
print(f"Expires: {payload.get('exp')}")

### 5.3: Use the token to get the list of available tools from AgentCore Gateway

In [ ]:
def list_tools(gateway_url, access_token):
    response = requests.post(
        gateway_url,
        headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {access_token}"
        },
        json={
            "jsonrpc": "2.0",
            "id": "list-tools-request",
            "method": "tools/list"
        }
    )
    return response.json()

tools = list_tools(gateway_url, access_token)
print(json.dumps(tools, indent=2))

### 5.4: Create an MCP Client and list available tools

In [ ]:
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp import MCPClient

# Set up MCP client
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway_url,
        headers={"Authorization": f"Bearer {access_token}"},
    )
)

mcp_client.start()
mcp_client.list_tools_sync()

### 5.5: Interacting with the Agent

Initialize the Agent with MCP Tools

In [ ]:
from strands import Agent
agent = Agent(tools=mcp_client.list_tools_sync())

In [ ]:
agent("What is the weather in Vancouver?")

In [ ]:
agent("Give me directions to Vancouver?")

## Conclusion and Cleanup

### Resource Cleanup

Clean up the resources created in this tutorial:

In [ ]:
import time

# 1. Delete lambda target on your Gateway
print("🗑️ Deleting gateway target...")
try:
    gateway_client.delete_gateway_target(gatewayIdentifier=gateway_id, targetId=target_id)
    print(f"✅ Target {target_id} deleted.")
    # Wait for detachment only if a deletion actually happened
    print("⏳ Waiting for detachment...")
    time.sleep(10)
except gateway_client.exceptions.ResourceNotFoundException:
    print(f"ℹ️ Target {target_id} already gone. Skipping...")

# 2. Delete gateway
print("🗑️ Deleting gateway...")
try:
    gateway_client.delete_gateway(gatewayIdentifier=gateway_id)
    print(f"✅ Gateway {gateway_id} deleted.")
except gateway_client.exceptions.ResourceNotFoundException:
    print(f"ℹ️ Gateway {gateway_id} not found. Skipping...")

# 3. Delete lambda function
print("🗑️ Deleting lambda function...")
try:
    function_name = lambda_arn.split(':')[-1]
    lambda_client.delete_function(FunctionName=function_name)
    print(f"✅ Lambda {function_name} deleted.")
except lambda_client.exceptions.ResourceNotFoundException:
    print("ℹ️ Lambda already deleted.")

# 4. Delete created roles
print("🗑️ Deleting IAM roles...")
for current_role_arn in [lambda_role_arn, role_arn]:
    try:
        role_name = current_role_arn.split('/')[-1]
        inline = iam_client.list_role_policies(RoleName=role_name)
        for policy_name in inline['PolicyNames']:
            iam_client.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
        iam_client.delete_role(RoleName=role_name)
        print(f"✅ Role {role_name} deleted.")
    except iam_client.exceptions.NoSuchEntityException:
        print("ℹ️ Role already deleted.")

print("✨ Cleanup finished.")

### Conclusion

This notebook demonstrated how to:
1. **Setup PingOne for Gateway** - Configure a PingOne resource and application specifically for the `mcp:invoke` scope.
2. **Provision AgentCore Gateway** - Deploy a gateway instance with a `CUSTOM_JWT` authorizer to trust PingOne tokens.
3. **Register Lambda Tools** - Link existing AWS Lambda functions as MCP targets within the gateway infrastructure.
4. **Implement Client-Side Auth** - Programmatically fetch and decode PingOne JWTs to authenticate MCP client requests.
5. **Orchestrate with Strands** - Connect a Strands Agent to the secure gateway to perform end-to-end tool invocation.

### Key Learnings:

- **Gateway Inbound Auth:** PingOne acts as a robust gatekeeper, ensuring only requests with valid, signed JWTs can access your MCP server.
- **MCP Protocol abstraction:** The gateway provides a uniform Model Context Protocol (MCP) interface, hiding the complexity of underlying Lambda or API logic.
- **JWT Token Validation:** AgentCore automatically validates PingOne JWT tokens
- **Security:** Unauthenticated requests are automatically rejected

### Next Steps:

- **Implement Outbound Auth:** Configure the gateway to securely connect to backend APIs using OAuth-based flows.
- **Expand the Toolset:** Add more sophisticated Lambda functions or OpenAPI-compliant APIs to the gateway’s target list.
- **Advanced Scoping:** Implement fine-grained access control by defining multiple MCP scopes for different agent personalities.